In [2]:
#For the dashboard 
import param
import panel as pn
import pandas as pd
import time
import requests

from io import StringIO


import plotly.express as px

pn.extension('plotly', 'tabulator', sizing_mode="stretch_width")

In [3]:
# For transcription 
import requests
from IPython.display import JSON
import numpy as np 
import time

# Visualization
from wordcloud import WordCloud, STOPWORDS
import plotly.express as px

# Panel/hvplot (holoviz)
import panel as pn
pn.extension()
import param
import hvplot.pandas

# Others
import pickle
from io import StringIO

In [17]:
class AudioDashboard(param.Parameterized):
    data = param.Dict()
  
    file_input = param.Parameter()
    
    #Setting the format of the dashboard 
    def __init__(self, **params):
        super().__init__(file_input=pn.widgets.TextInput(), **params)
        self.audioplay = pn.pane.Audio( name='Audio')  
        self.transcript = pn.pane.Str(max_width = 600, height_policy = "fit")
        self.wordcloud = pn.panel()
        self.sentimentplot = pn.Column()
        self.autohighlights = pn.Row()
#        self.autochapters = pn.Row()
        self.download = pn.widgets.FileDownload(name="Download transcript", filename="transcript.txt", callback=self._download_callback, button_type="primary")

# --------------------------------------------------------------------------------------------------------------------
       
    #Get and analyze transcript 
    def get_and_analyze_transcript(self):
        audio_url = self.file_input.value
                
        if audio_url:
            print("yessss")
            API_KEY = "4f6f0c7f212343d6b745191a8652340b"

            endpoint = "https://api.assemblyai.com/v2/transcript"
            audio_url_json = {
                "audio_url": audio_url,
                "sentiment_analysis": True,
                "auto_highlights": True,
                "iab_categories": True,
                "auto_chapters": True
            }
            headers = {
                "authorization": API_KEY,
                "content-type": "application/json"
            }
            transcript_response = requests.post(endpoint, json=audio_url_json, headers=headers)

            result_endpoint = endpoint + '/' + transcript_response.json()["id"]
            headers_auth = {
                "authorization": API_KEY,
            }
            response = requests.get(result_endpoint, headers=headers_auth)
            while response.json()['status'] != "completed":
                response = requests.get(result_endpoint, headers=headers_auth)
                time.sleep(3)
                
            return response.json()

        else:
            print("no")
            return
        
        
# -------------------------------------------------------------------------------------------------------------------- 
    #Parsing the input data in to analyze 
    @pn.depends("file_input.value", watch=True)
    def _parse_file_input(self):
        print("parse input ")
        print(self.file_input)
        value = self.file_input.value
        if value:
            print("in parse")
            self.data = self.get_and_analyze_transcript()
            print(self.data)
        else:
            print("error")
            
            
    #Download callback 
    def _download_callback(self):
        print("download")
        if self.data is not None:
            buffer = StringIO()
            buffer.write(self.data["text"])
            buffer.seek(0)
            print("return buffer")

            return buffer  
        else:
            return
        
    #Polling response    
    def poll(self):
        polling_endpoint = transcript_endpoint + '/' + self.transcript_id
        polling_response = requests.get(polling_endpoint, headers=self.headers)
        self.data = polling_response.json()

# --------------------------------------------------------------------------------------------------------------------

    #Get the transcript------------------------------------------ 
    @pn.depends('data', watch=True)
    def get_transcript(self):
        print("get transcript ")
        self.transcript.object = self.data["text"] 
        
    #Get the audio----------------------------------------------- 
    @pn.depends('data', watch=True)
    def get_audio(self):
        print("get audio ")
        self.audioplay.object = self.file_input.value
    
   
    #Get the wordcloud------------------------------------
    @pn.depends('data', watch=True)
    def get_wordcloud(self):
        print("get wordcloud ")
        stopwords = set(STOPWORDS)
        transcript = self.data["text"]
        transcript_lower = [item.lower() for item in str(transcript).split()]
        all_words = ' '.join(transcript_lower)     
        wordcloud = WordCloud(background_color='black', stopwords = stopwords, max_words = 20,
                            colormap='viridis', collocations=False).generate(all_words)
        wordcloud_plot = px.imshow(wordcloud) 
        wordcloud_plot.update_xaxes(showticklabels=False)
        wordcloud_plot.update_yaxes(showticklabels=False)
        self.wordcloud.object = pn.Pane(wordcloud_plot)
        
    
    
    #Get the sentiment plot-------------------------------------
    @pn.depends('data', watch=True)
    def get_sentimentplot(self):
        print("get sentimentplot ")
        sentiment = self.data["sentiment_analysis_results"]
        sentiment_df = pd.DataFrame(sentiment)
        sentiment_df_grouped = sentiment_df['sentiment'].value_counts()
        sentiment_plot = sentiment_df_grouped.hvplot(title = "Sentences by Sentiment Category", kind="bar")
        positive_df = sentiment_df[sentiment_df["sentiment"] == "POSITIVE"][["text", "sentiment"]]
        negative_df = sentiment_df[sentiment_df["sentiment"] == "NEGATIVE"][["text", "sentiment"]]
        neutral_df = sentiment_df[sentiment_df["sentiment"] == "NEUTRAL"][["text", "sentiment"]]
        sentiment_tabs =  pn.Tabs(('Sentiment overview', sentiment_plot), 
                                     ('Positive', pn.widgets.DataFrame(positive_df, autosize_mode='fit_columns', width=700, height=300)),
                                     ('Negative', pn.widgets.DataFrame(negative_df, autosize_mode='fit_columns', width=700, height=300)),
                                     ('Neutral', pn.widgets.DataFrame(neutral_df, autosize_mode='fit_columns', width=700, height=300)))
        self.sentimentplot.object  = sentiment_tabs 
    
    #Get the autochapters --------------------------------------
    @pn.depends('data', watch=True)
    def get_autochapters(self):
        print("get chapters ")
        self.chapters.object = self.data["chapters"]
    
#   #Create the chapter summary layout     
#   chapters_layout = pn.Column(pn.pane.Markdown("### Auto Chapter Summary"))
#
#   class ButtonAudio():
#       def __init__(self, start_time):
#           self.start_time = start_time
#           self.button = pn.widgets.Button(name=str(int(self.start_time/1000)), button_type='primary', width=60)
#           self.chapter_audio = pn.pane.Audio(self.file_input.value, name='Audio', time = round(self.start_time/1000))
#           self.button.on_click(self.move_audio_head)
#       def move_audio_head(self, event):
#           self.chapter_audio.time = self.start_time/1000
#      
#   for chapter in s:
#       chapter_summary = pn.widgets.StaticText(value=chapter["summary"], width=1000, height_policy = "fit")
#       button_audio = ButtonAudio(chapter["start"])
#       button = button_audio.button
#       chapter_audio = button_audio.chapter_audio
#       chapters_layout.append(pn.Row(pn.Column(button), pn.Column(chapter_audio), pn.Column(chapter_summary)))
#       
#       self.autochapters.object = chapters_layout
        
    
    #Get the autohighlights------------------------------------
    @pn.depends('data', watch=True)
    def get_autohighlights(self):
        print("get highlights ")
        hightlights = self.data["auto_highlights_result"]["results"]
        highlights_df = pd.DataFrame(highlights)
        highlights_df_grouped = highlights_df.groupby(["count","rank"])['text'].apply(",\n". join).reset_index()
        highlights_plot = highlights_df_grouped.hvplot.points(x="count", y = "rank",
                                                     padding =0.4, hover_cols= 'all', 
                                                     width =1300, height = 600, 
                                                     size= 50, title = 'Automatic Highlighted Words') *\
        highlights_df_grouped.hvplot.labels(x="count", y = "rank",
                                           text ="text", text_baseline ='top',
                                           hover= False).opts(fontscale=1.5)
        
        self.autohighlights.object = highlights_plot

    
# --------------------------------------------------------------------------------------------------------------------
    #Final view at the end     
    def view(self):
        return pn.Column(
            "## Wordcloud", 
            self.wordcloud, 
            "## Sentiment plot",
            self.sentimentplot,
            "## Auto highlights",
            self.autohighlights,
 #           '## Auto Chaprers,
 #           self.autochapters, 
            "## Audio",
            self.audioplay,
            "## Transcript",
            self.transcript,

        )
    
audio_app = AudioDashboard()

audio_app_view = audio_app.view()
audio_app_view


TypeError: panel() missing 1 required positional argument: 'obj'

In [13]:
description = """
This is my app :)
"""

component = pn.Column(
    description,
    audio_app_view,
    sizing_mode='stretch_both'
)
component

template = pn.template.FastListTemplate(
    title='Audio Content Explorer', 
    sidebar=[ pn.pane.Markdown("# Explore audio content"), 
             pn.pane.Markdown("#### This app analyzes the content of your audio file, including sentiment, wordcloud, automatic content summary and highlights using AssemblyAI API."),
             pn.pane.Markdown("### Input a link:"), 
             audio_app.file_input, 
             audio_app.download],
    main=[component],
    accent_base_color="#88d8b0",
    header_background="#88d8b0",
)

template.servable()

Traceback (most recent call last):
 File "C:\Users\taylo\anaconda3\lib\site-packages\pyviz_comms\__init__.py", line 346, in _handle_msg
 self._on_msg(msg)
 File "C:\Users\taylo\anaconda3\lib\site-packages\panel\viewable.py", line 292, in _on_msg
 doc.unhold()
 File "C:\Users\taylo\anaconda3\lib\site-packages\bokeh\document\document.py", line 799, in unhold
 self.callbacks.unhold()
 File "C:\Users\taylo\anaconda3\lib\site-packages\bokeh\document\callbacks.py", line 396, in unhold
 self.trigger_on_change(event)
 File "C:\Users\taylo\anaconda3\lib\site-packages\bokeh\document\callbacks.py", line 373, in trigger_on_change
 invoke_with_curdoc(doc, event.callback_invoker)
 File "C:\Users\taylo\anaconda3\lib\site-packages\bokeh\document\callbacks.py", line 408, in invoke_with_curdoc
 return f()
 File "C:\Users\taylo\anaconda3\lib\site-packages\bokeh\util\callback_manager.py", line 191, in invoke
 callback(attr, old, new)
 File "C:\Users\taylo\anaconda3\lib\site-packages\panel\reactive.py", line 392, in _comm_change
 self._schedule_change(doc, comm)
 File "C:\Users\taylo\anaconda3\lib\site-packages\panel\reactive.py", line 376, in _schedule_change
 self._change_event(doc)
 File "C:\Users\taylo\anaconda3\lib\site-packages\panel\reactive.py", line 370, in _change_event
 self._process_events(events)
 File "C:\Users\taylo\anaconda3\lib\site-packages\panel\reactive.py", line 315, in _process_events
 self.param.update(**self_events)
 File "C:\Users\taylo\anaconda3\lib\site-packages\param\parameterized.py", line 1898, in update
 self_._batch_call_watchers()
 File "C:\Users\taylo\anaconda3\lib\site-packages\param\parameterized.py", line 2059, in _batch_call_watchers
 self_._execute_watcher(watcher, events)
 File "C:\Users\taylo\anaconda3\lib\site-packages\param\parameterized.py", line 2021, in _execute_watcher
 watcher.fn(*args, **kwargs)
 File "C:\Users\taylo\anaconda3\lib\site-packages\param\parameterized.py", line 669, in caller
 return function()
 File "C:\Users\taylo\anaconda3\lib\site-packages\param\parameterized.py", line 407, in _depends
 return func(*args, **kw)
 File "<ipython-input-12-d56cc0dc8f95>", line 66, in _parse_file_input
 self.data = self.get_and_analyze_transcript()
 File "C:\Users\taylo\anaconda3\lib\site-packages\param\parameterized.py", line 369, in _f
 return f(self, obj, val)
 File "C:\Users\taylo\anaconda3\lib\site-packages\param\parameterized.py", line 1248, in __set__
 obj.param._call_watcher(watcher, event)
 File "C:\Users\taylo\anaconda3\lib\site-packages\param\parameterized.py", line 2039, in _call_watcher
 self_._execute_watcher(watcher, (event,))
 File "C:\Users\taylo\anaconda3\lib\site-packages\param\parameterized.py", line 2021, in _execute_watcher
 watcher.fn(*args, **kwargs)
 File "C:\Users\taylo\anaconda3\lib\site-packages\param\parameterized.py", line 669, in caller
 return function()
 File "C:\Users\taylo\anaconda3\lib\site-packages\param\parameterized.py", line 407, in _depends
 return func(*args, **kw)
 File "<ipython-input-12-d56cc0dc8f95>", line 144, in get_autochapters
 self.chapters.object = self.data["chapters"]
AttributeError: 'AudioDashboard' object has no attribute 'chapters'
 

download 
return buffer

FastListTemplate
    [js_area] HTML(None, height=0, margin=0, sizing_mode='fixed', width=0)
    [actions] TemplateActions()
    [busy_indicator] LoadingSpinner(height=20, width=20)
    [1907004917024] Column(sizing_mode='stretch_both')
        [0] Markdown(str, sizing_mode='stretch_width')
        [1] Column(sizing_mode='stretch_width')
            [0] Markdown(str, sizing_mode='stretch_width')
            [1] Row(sizing_mode='stretch_width')
            [2] Markdown(str, sizing_mode='stretch_width')
            [3] Column(sizing_mode='stretch_width')
            [4] Markdown(str, sizing_mode='stretch_width')
            [5] Row(sizing_mode='stretch_width')
            [6] Markdown(str, sizing_mode='stretch_width')
            [7] Audio(None, sizing_mode='stretch_width', volume=100)
            [8] Markdown(str, sizing_mode='stretch_width')
            [9] Str(None, height_policy='fit', max_width=600, sizing_mode='stretch_width')
    [1908993807504] Markdown(str, sizing_mode='stretch_width')
    [1906972877728] Markdown(str, sizing_mode='stretch_width')
    [1907004917552] Markdown(str, sizing_mode='stretch_width')
    [1907004899392] TextInput(sizing_mode='stretch_width')
    [1907004921984] FileDownload(button_type='primary', callback=<bound method AudioDashboa..., filename='transcript.txt', label='Download transcript.txt', name='Download transcript', sizing_mode='stretch_width')

In [ ]:
https://github.com/thu-vu92/audio_analyzer_assemblyai/blob/main/How_I_Would_Learn_to_Code.mp3?raw=true